In [1]:
import cv2
import numpy as np
from datetime import datetime
import sys
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QPalette, QColor,QFont,QImage, QPixmap
from PyQt5.QtWidgets import QApplication,QAction, QLabel,QMainWindow, QWidget, QVBoxLayout, QHBoxLayout, QPushButton, QMessageBox, QFileDialog

In [4]:
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        # Initialize the image variables
        self.path = None
        self.image = None
        self.original = None
        self.history = []
        self.previous = None

        # Create the menu bar
        menu_bar = self.menuBar()

        # Create the file menu and actions
        file_menu = menu_bar.addMenu('File')
        load_action = QAction('Load Image', self)
        load_action.triggered.connect(self.load_image)
        file_menu.addAction(load_action)

        # Create the view menu and actions
        view_menu = menu_bar.addMenu('View')
        display_action = QAction('Display Image With Time', self)
        display_action.triggered.connect(self.display_image_with_time)
        view_menu.addAction(display_action)

        # Create the image manipulation menu and actions
        Image_Manipulation_menu = menu_bar.addMenu('Image Manipulation')
        Image_Manipulation_action = QAction('Image Manipulation', self)
        Image_Manipulation_action.triggered.connect(self.Image_Manipulation)
        Image_Manipulation_menu.addAction(Image_Manipulation_action)

        # Create the main widget and layout
        central_widget = QWidget(self)
        layout = QVBoxLayout(central_widget)
        layout.setAlignment(Qt.AlignCenter)

        # Create the label and buttons
        label = QLabel('Interactive Image Processing with Mouse Events and Time Display')
        label_font = QFont('Arial', 14, QFont.Bold)
        label.setFont(label_font)

        load_image_button = QPushButton('Load Image', self)
        load_image_button.clicked.connect(self.load_image)
        load_image_button.setStyleSheet('QPushButton {background-color: #6d6875; color: white; font-weight: bold; padding: 10px;}')

        display_image_button = QPushButton('Display Image With Time', self)
        display_image_button.clicked.connect(self.display_image_with_time)
        display_image_button.setStyleSheet('QPushButton {background-color: #b5838d; color: white; font-weight: bold; padding: 10px;}')

        Image_Manipulation_button = QPushButton('Image Manipulation', self)
        Image_Manipulation_button.clicked.connect(self.Image_Manipulation)
        Image_Manipulation_button.setStyleSheet('QPushButton {background-color: #ffb4a2; color: white; font-weight: bold; padding: 10px;}')

        image_parameters_button = QPushButton('Adjusting Image Parameters', self)
        image_parameters_button.clicked.connect(self.adjusting_image_parameters)
        image_parameters_button.setStyleSheet('QPushButton {background-color: #ffcdb2; color: white; font-weight: bold; padding: 10px;}')

        help_button = QPushButton('Help', self)
        help_button.clicked.connect(self.show_help)
        help_button.setStyleSheet('QPushButton {background-color: #b2d8d8; color: white; font-weight: bold; padding: 10px;}')

        # Add the label and buttons to the layout
        layout.addWidget(label)
        layout.setSpacing(60)
        button_layout = QHBoxLayout()
        button_layout.addWidget(load_image_button)
        button_layout.addWidget(display_image_button)
        button_layout.addWidget(Image_Manipulation_button)
        button_layout.addWidget(image_parameters_button)
        button_layout.addWidget(help_button)
        layout.addLayout(button_layout)

        # Set the main widget as the central widget
        self.setCentralWidget(central_widget)

        # Set the background color of the main widget
        palette = QPalette()
        palette.setColor(QPalette.Background, QColor(255,255,224))
        self.centralWidget().setAutoFillBackground(True)
        self.centralWidget().setPalette(palette)
        
    def show_help(self):
        QMessageBox.information(self, "What is the Purpose of a Program?",'''
       

1. "Display Image With Time":
   - Display the loaded image with current time.
   - Allow the user to draw a rectangle by pressing and dragging the left mouse button.
   - Allow the user to draw a circle by pressing and dragging the right mouse button.
   - Allow the user to perform image translation by pressing and dragging the middle mouse button.
   - Allow the user to reset the image to its original state by pressing the 'r' key.
   - Allow the user to exit the program by pressing the 'q' key.

2. "Image Manipulation":
   - Allow the user to convert the image to grayscale by pressing the 'g' key.
   - Allow the user to reset the image to its original state by pressing the 'r' key.
   - Allow the user to save the image to disk by pressing the 's' key.
   - Allow the user to crop the region of interest defined by the rectangle using array slicing by pressing the 'c' key.
   - Allow the user to undo the previous operation by pressing the 'z' key.
   - Allow the user to display a help message by pressing the 'h' key.
   - Allow the user to exit the program by pressing the 'q' key.

3. "Adjusting Image Parameters":
   - Allow the user to adjust the brightness,contrast,saturation level of the image using a trackbar.
  
        
        
        
        ''')
    def load_image(self):
        # Open a file dialog to select an image file
        path, _ = QFileDialog.getOpenFileName(self, 'Open Image', '', 'Image Files (*.png *.jpg *.jpeg *.bmp)')

        # Load the image file and store its path and original copy
        if path:
            self.path = path
            self.original = cv2.imread(path)
            self.reset_the_image()
            self.statusBar().showMessage(f'Image loaded successfully: {path}')

    def reset_the_image(self):
        # Reset the image to the original copy
        self.image = self.original.copy()
        self.previous = self.image.copy()
        self.history = []

    def mouse_callback(self, event, x, y, flags, param):
        # Process the mouse events on the image
        if event == cv2.EVENT_LBUTTONDOWN:
            cv2.rectangle(self.image, (x, y), (x + 10, y + 10),(255,225,85), -1)

        elif event == cv2.EVENT_RBUTTONDOWN:
            cv2.circle(self.image, (x, y), 20, (0,255,255), -1)

        elif event == cv2.EVENT_MBUTTONDOWN:
            rows, cols, ch = self.image.shape
            pts1 = np.float32([[50, 50],
                   [200, 50],
                   [50, 200]])
 
            pts2 = np.float32([[10, 100],
                   [200, 50],
                   [100, 250]])
 
            M = cv2.getAffineTransform(pts1, pts2)
            self.image = cv2.warpAffine(self.image, M, (cols, rows))

    def display_image_with_time(self):
        # Display the image with time and mouse events
        if not self.image is None:
            cv2.namedWindow('image')
            cv2.setMouseCallback('image', self.mouse_callback)

            while True:
                current_time = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
                image_with_time = self.image.copy()
                cv2.putText(image_with_time, str(current_time), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)        
                cv2.imshow('image', image_with_time)
                key = cv2.waitKey(1) & 0xFF

                if key == ord('q'):
                    break
                elif key == ord('r'):
                    self.reset_the_image()

            cv2.destroyAllWindows()
        else:
            self.statusBar().showMessage('No image loaded!')
            
    def adjusting_image_parameters(self):
        
        def change_image(x):
            Brightness = cv2.getTrackbarPos('Brightness', 'image')
            Contrast = cv2.getTrackbarPos('Contrast', 'image')
            Saturation = cv2.getTrackbarPos('Saturation', 'image')
            
            adjusted_image = cv2.convertScaleAbs(self.original, alpha=Contrast/100, beta=Brightness)

            hsv_image = cv2.cvtColor(adjusted_image, cv2.COLOR_BGR2HSV)
            hsv_image[:, :, 1] = np.clip(hsv_image[:, :, 1] + Saturation, 0, 255)
            adjusted_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)
            

            cv2.imshow('image', adjusted_image)

        cv2.namedWindow('image')
        cv2.createTrackbar('Brightness', 'image', 0, 100, change_image)
        cv2.createTrackbar('Contrast', 'image', 0, 100, change_image)
        cv2.createTrackbar('Saturation', 'image', 0, 100, change_image)
        

        change_image(0)

        while True:
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break

        cv2.destroyAllWindows()
        
    def undo_the_previous_operation(self):
        # Undo the last image processing operation
        if self.history:
            self.image = self.history.pop()
        else:
            print('This is the original image')

    def display_a_help_message(self):
        # Display the help message with the available image processing operations
        help_message = "g - Convert the image to grayscale\n" \
                        "r - Reset the image to its original state\n" \
                        "s - Save the image to disk\n" \
                        "c - Crop the region of interest defined by the rectangle using array slicing\n" \
                        "z - Undo the previous operation\n" \
                        "h - Display a help message\n" \
                        "q - Exit the program."
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.5
        thickness = 1
        color = (255, 255, 255)
        x = 10
        y = 20
        lines = help_message.split('\n')
        for i, line in enumerate(lines):
            cv2.putText(self.image, line, (x, y + i*20), font, font_scale, color, thickness)
    
    def Image_Manipulation(self):
        # Apply image Manipulation operations on the loaded image
        if not self.image is None:
            cv2.namedWindow('image')
            
            
            while True:
                cv2.imshow('image', self.image)
                key = cv2.waitKey(1) & 0xFF

                if key == ord('g'):
                    self.history.append(self.image.copy())
                    self.image = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)

                elif key == ord('r'):
                    self.reset_the_image()

                elif key == ord('s'):
                    cv2.imwrite('image1.jpg', self.image)
                    self.statusBar().showMessage('Image saved successfully!')

                elif key == ord('c'):
                    self.history.append(self.image.copy())
                    self.image = self.image[400:1000, 100:500]

                elif key == ord('z'):
                    self.undo_the_previous_operation()

                elif key == ord('h'):
                    self.display_a_help_message()

                elif key == ord('q'):
                    break
            
            cv2.destroyAllWindows()
            
                    
        else:
            self.statusBar().showMessage('No image loaded!')        
        

In [ ]:
if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())